In [1]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import pandas as pd
import numpy as np
import re
import string
import nltk

pretrained= "mdhugol/indonesia-bert-sentiment-classification"

model = AutoModelForSequenceClassification.from_pretrained(pretrained)
tokenizer = AutoTokenizer.from_pretrained(pretrained)

sentiment_analysis = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

In [2]:
# Load Processed Survey Data
cleaned_survey = pd.read_csv('./data/survey_clean_stemmed.csv')
cleaned_survey.head()

label_index = {'LABEL_0': 'positive', 'LABEL_1': 'neutral', 'LABEL_2': 'negative'}

In [3]:
# Iterate over each row in the dataframe
for index, row in cleaned_survey.iterrows():
    try:
        # Get the text from the row
        text = row["stemmed_message"]
        # Classify the text and get the label
        result = sentiment_analysis(text)
        status = label_index[result[0]["label"]]
        score = result[0]["score"]

        # Add the label and score to the dataframe
        cleaned_survey.at[index, "sentiment"] = status
        cleaned_survey.at[index, "sentiment_score"] = score
    except Exception as e:
        print(f"Error at index {index}: {e}")
        continue

In [4]:
import os

# Define the file path
file_path = "./data/survey_clean_stemmed_sentiment_indoBERT.csv"

# Check if the directory exists
directory = os.path.dirname(file_path)
if not os.path.exists(directory):
    os.makedirs(directory)

# Try to write the file
try:
    cleaned_survey.to_csv(file_path, index=False)
    print(f"File saved successfully to {file_path}")
except Exception as e:
    print(f"Failed to write file to {file_path}. Error: {e}")

File saved successfully to ./data/survey_clean_stemmed_sentiment_indoBERT.csv
